## 📝 Descripción del proceso

Este script permite automatizar el registro de intervenciones en documentos `.docx` utilizando fragmentos de texto resaltado y/o tachado. A partir de documentos previamente anotados con colores específicos, se extrae información estructurada que puede luego analizarse o clasificarse.

### 🔄 Pasos principales:

1. **Leer los documentos Word**  
   Se recorren todos los archivos `.docx` dentro de una carpeta determinada.

2. **Identificar fragmentos resaltados y tachados**  
   El script detecta tanto el texto que ha sido resaltado con color como aquel que ha sido marcado con tachado.

3. **Extraer el color del resaltado**  
   Se accede directamente al XML del archivo para obtener el color de fondo de cada fragmento resaltado (en formato hexadecimal).

4. **Clasificar los fragmentos**  
   A partir de un diccionario de colores predefinido, cada intervención se asocia a una categoría (por ejemplo, "REEMPLAZO", "AGREGADO", etc.).

5. **Almacenar los datos en una tabla (DataFrame)**  
   Cada fragmento se guarda con su información relevante (texto, color, tipo de intervención, etc.) en un DataFrame de `pandas`.

6. **Exportar la tabla a CSV o visualizarla**  
   La tabla resultante puede exportarse como archivo `.csv` o imprimirse en pantalla para su posterior revisión y análisis.

> ℹ️ Este script fue creado como parte de una tesina y responde a necesidades concretas de sistematización y análisis textual.


> ⚠️ **Importante aclaración sobre el código**  
> El código desarrollado es una **primera aproximación** que tiene como objetivo aprender a trabajar con XML y a procesar diferentes **elementos textuales** de manera automatizada. Si bien cumple con la función principal, no está exento de errores.  
> Durante el proceso, se realizó una **revisión manual de los datos** para corregir las posibles inconsistencias y asegurar que todo estuviera registrado correctamente, especialmente en casos donde el script no pudo identificar con precisión ciertos elementos.  
> El propósito de este proyecto fue doble: por un lado, aprender a automatizar tareas repetitivas relacionadas con el análisis de texto y, por otro, desarrollar una herramienta que agilizara mi trabajo de análisis en la tesina, profundizando al mismo tiempo en técnicas de **Procesamiento de Lenguaje Natural** (PNL).

In [ ]:
# 📚 Importación de librerías necesarias

# Para trabajar con documentos Word (.docx)
from docx import Document
from docx.oxml.ns import qn

# Para manejar estructuras tabulares
import pandas as pd

# Para interactuar con el sistema de archivos
import os

# Para trabajar con expresiones regulares
import re


In [ ]:
# 📂 Ruta de la carpeta con los documentos .docx (modificar según el entorno)
folder = "./versiones"

# 🧾 Lista para almacenar la información extraída de cada documento
resultados = []

# 🎨 Conjunto para registrar todos los colores de resaltado encontrados
colores_resaltado = set()

# 🔍 Esto se hace como paso previo para detectar todos los colores efectivamente presentes en los documentos.
# En el etiquetado manual con Google Docs, puede haber confusiones: por ejemplo, se usó "naranja" para los REEMPLAZOS,
# pero podrían haberse aplicado distintos tonos de naranja con códigos hexadecimales levemente distintos.
# Por eso, primero se extraen todos los colores reales usados en los documentos para garantizar que ninguno quede 
# sin clasificar.


In [ ]:
# 🧪 Procesamiento inicial de documentos para detectar colores de resaltado utilizados

def procesar_archivo(file_path, filename):
    """
    Procesa un archivo .docx, detecta los fragmentos de texto resaltados,
    extrae el color exacto del resaltado y lo guarda junto con otra información relevante.
    """
    doc = Document(file_path)
    current_version = None  # Guarda la versión actual (por ejemplo: "V1", "T2", etc.)

    for para in doc.paragraphs:
        texto_para = para.text.strip()

        # Detectar si el párrafo indica una versión (V1–V6 o T2–T5)
        if re.match(r"^(V[1-6]|T[2-5])$", texto_para):
            current_version = texto_para
            continue

        # Recorrer los fragmentos (runs) del párrafo para encontrar resaltados
        for run in para.runs:
            shading_fill = None
            rpr = run._element.find(qn('w:rPr'))  # Accede a las propiedades del run (rPr)
            if rpr is not None:
                shd = rpr.find(qn('w:shd'))  # Busca la propiedad de sombreado
                if shd is not None:
                    shading_fill = shd.get(qn('w:fill'))  # Extrae el código de color hexadecimal

            if shading_fill:
                color = shading_fill
                colores_resaltado.add(color)
                resultados.append({
                    "archivo": filename,
                    "versión": current_version,
                    "color_resaltado": color,
                    "texto": run.text.strip()
                })

# 🚀 Ejecutar el procesamiento sobre todos los archivos de la carpeta

for filename in os.listdir(folder):
    # Procesar solo archivos .docx que sigan la nomenclatura E00, E01, ..., E19 (excepto E08)
    if filename.endswith(".docx") and re.match(r"E\d+", filename):
        if filename.startswith("E08"):
            continue  # E08 se omite por decisión metodológica
        file_path = os.path.join(folder, filename)
        procesar_archivo(file_path, filename)

# 🖨️ Mostrar los colores de resaltado encontrados (para luego clasificarlos)
print("🎨 Colores de resaltado encontrados:")
for color in colores_resaltado:
    print(" -", color)

# 🧾 Mostrar un resumen de los fragmentos extraídos
print("\n📋 Detalles extraídos:")
for r in resultados:
    print(r)


Colores de resaltado encontrados:
 - 8e7cc3
 - f6b26b
 - c27ba0
 - ffd966
 - e06666
 - 76a5af
 - 93c47d
 - auto
 - cccccc

Detalles extraídos:
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': 'f6b26b', 'texto': 'handling'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': '93c47d', 'texto': 'all of this has been carried out through the present enzymatic activity with ammonium sulfate'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': 'c27ba0', 'texto': 'vegetable'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': 'f6b26b', 'texto': 'At the same time'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': '76a5af', 'texto': 'p-Nitrophenol'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': '93c47d', 'texto': 'we used calibration curves'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': 'f6b26b', 'texto': 'made'}
{'archivo': 'E00.docx', 'versión': 'V2', 'color_resaltado': '76a5af', 'texto': 'standard'}
{'archivo': 'E00

⚠️ **Nota sobre la estructura del procesamiento**

Este script fue diseñado específicamente para analizar documentos utilizados en el marco de mi tesina. En dichos textos:

- Cada archivo se identifica con un código de experimento en formato `E##`, por ejemplo: `E00`, `E02`, `E13`, etc.  
- Dentro de los documentos, las diferentes versiones del texto se indican mediante encabezados como `V1`, `V2`, ..., `T2`, `T5`, etc., ubicados en párrafos independientes.

El código actual filtra los archivos `.docx` que siguen este patrón de nomenclatura (`E##`) y detecta las versiones internas a partir de esos marcadores (`V1` a `V6`, `T2` a `T5`).

👉 Si querés reutilizar o adaptar este código para trabajar con otro conjunto de documentos, es importante ajustar las expresiones regulares y las condiciones de filtrado según los nombres de archivo y los formatos de versionado que estés utilizando.


In [ ]:
# === Construcción del diccionario de colores ===

# Abrir el documento que contiene el mapeo entre nombres y colores de resaltado.
# Este documento fue preparado manualmente y debe contener un párrafo por color,
# con el nombre de la categoría como texto y el resaltado aplicado sobre ese texto.
doc = Document("colores.docx")

# Diccionario donde se almacenarán los pares {nombre: código_de_color}
diccionario_colores = {}

# Iterar por cada párrafo del documento
for para in doc.paragraphs:
    nombre_linea = para.text.strip()  # Se espera que cada línea contenga un nombre de color (ej. "naranja 1")
    if not nombre_linea:
        continue  # Saltar los párrafos vacíos, si los hubiera

    color_codigo = None  # Inicializamos el color por defecto como None

    # Recorrer los "runs" (fragmentos de texto con formato uniforme) del párrafo
    for run in para.runs:
        # Acceder al XML interno para buscar la propiedad de resaltado
        rpr = run._element.find(qn('w:rPr'))  # Parte del run que contiene propiedades de estilo
        if rpr is not None:
            shd = rpr.find(qn('w:shd'))  # Elemento que contiene información de sombreado (highlight)
            if shd is not None:
                color_codigo = shd.get(qn('w:fill'))  # Extraer el código hexadecimal del color
                break  # Suponemos que hay un solo color por párrafo, así que salimos del bucle

    # Asociar el nombre de color con su código en el diccionario
    diccionario_colores[nombre_linea] = color_codigo

# Mostrar el resultado para verificar que se haya construido correctamente
print("Diccionario de colores:")
for nombre, codigo in diccionario_colores.items():
    print(f"{nombre}: {codigo}")


Diccionario de colores:
rojo 1: ea9999
rojo 2: e06666
rojo 3: cc0000
naranja 1: f9cb9c
naranja 2: f6b26b
naranja 3: e69138
amarillo 1: ffe599
amarillo 2: ffd966
amarillo 3: f1c232
verde 1: b6d7a8
verde 2: 93c47d
verde 3: 6aa84f
azul 1: a2c4c9
azul 2: 76a5af
azul 3: 45818e
azul 4: a4c2f4
azul 5: 6d9eeb
azul 6: 3c78d8
azul 7: 9fc5e8
azul 8: 6fa8dc
azul 9: 3d85c6
violeta 1: b4a7d6
violeta 2: 8e7cc3
violeta 3: 674ea7
rosa 1: d5a6bd
rosa 2: c27ba0
rosa 3: a64d79
sin resaltado: None
gris 1: 999999
gris 2: cccccc
gris 3: b7b7b7


🎨 **Construcción del diccionario de colores**

Para poder interpretar automáticamente el significado de cada color de resaltado utilizado en los textos, se creó un documento auxiliar (`colores.docx`) con las etiquetas y colores utilizados durante el etiquetado manual. Este documento fue elaborado en Google Docs e incluye, por ejemplo:

- Un párrafo con el texto "rosa 1", resaltado con un tono de rosa.
- Otro con "naranja 2", resaltado con un tono distinto de naranja.
- Y así sucesivamente.

🔎 El objetivo de este archivo es mapear etiquetas como `"rosa 1"`, `"naranja 2"`, etc., a su correspondiente código hexadecimal de color (`f9cb9c`, `f4cccc`, etc.).

El código recorre cada párrafo del documento y extrae:
- El texto visible (que se usará como **clave** en el diccionario).
- El color del resaltado de fondo (como **valor** en el diccionario).

🧩 Este diccionario se usará más adelante para interpretar el significado de cada color encontrado en los textos analizados.

👉 Si querés adaptar esta herramienta a otro conjunto de colores o etiquetas, podés modificar el archivo `colores.docx` con los nuevos nombres y resaltados. Asegurate de mantener un formato similar: un párrafo por color, con el texto representando su etiqueta.


> ⚠️ **Nota explicativa**  
> Esta comparación de códigos hexadecimales fue realizada con asistencia de ChatGPT.  
> Distinguir manualmente entre códigos tan similares —como `ffe599` y `ffd966`— puede resultar confuso, ya que sus diferencias visuales son mínimas.  
> Una IA como ChatGPT es especialmente útil en estos casos, ya que puede identificar con precisión secuencias de caracteres parecidas pero no idénticas.  
> Esto fue clave para asegurar que los colores utilizados en los resaltados coincidieran exactamente con los valores definidos en el diccionario.

```
Comparé los códigos usando ChatGPT (para no confundirme):

Colores encontrados que están en tu diccionario:
b7b7b7 → gris 3
9fc5e8 → azul 7
3d85c6 → azul 9
ea9999 → rojo 1
f9cb9c → naranja 1
ffe599 → amarillo 1
b4a7d6 → violeta 1
999999 → gris 1
a4c2f4 → azul 4
93c47d → verde 2
f6b26b → naranja 2
ffd966 → amarillo 2
```

```
Colores encontrados que NO están en el diccionario:
ffe6e6 → Parece ser una variante más clara de rojo, pero no está en tu lista.
df92ba → No coincide con los tonos de violeta que tienes; parece un rosa o violeta claro.
d5a6bd → Similar al anterior, podría ser una variante de violeta o rosa.
auto → Esto no es un color real, puede referirse a un resaltado automático o sin color definido.
00b0f0 → No está en tu lista; es un azul cian brillante.
b08cce → Podría ser un violeta más claro, pero no coincide con los de tu lista
```


In [ ]:
"La idea de este bloque es identificar los colores en los textos que no estaban en el diccionario que yo había definido"

# Definir los colores desconocidos que se desean encontrar
colores_a_buscar = {"ffe6e6", "df92ba", "d5a6bd", "00b0f0", "b08cce"}

# Crear diccionario para registrar los colores encontrados junto con su contexto
colores_encontrados = {}

# Definir espacio de nombres XML correspondiente al esquema de WordprocessingML
NAMESPACE = "{http://schemas.openxmlformats.org/wordprocessingml/2006/main}"

def procesar_archivo(file_path, filename):
    """
    Procesar un archivo .docx para buscar colores específicos y registrar su contexto
    """
    global colores_a_buscar, colores_encontrados
    
    doc = Document(file_path)
    current_version = None  # Almacenar la versión actual del documento (ej. "V1", "T2", etc.)

    for para in doc.paragraphs:
        texto_para = para.text.strip()

        # Detectar líneas de versión (V1-V6, T2-T5) y actualizar el contexto
        if re.match(r"^(V[1-6]|T[2-5])$", texto_para):
            current_version = texto_para
            continue

        # Recorrer cada run (fragmento de texto con formato) del párrafo
        for run in para.runs:
            shading_fill = None

            # Buscar propiedades de formato del run
            rpr = run._element.find(f"{NAMESPACE}rPr")
            if rpr is not None:
                shd = rpr.find(f"{NAMESPACE}shd")
                if shd is not None:
                    shading_fill = shd.get(f"{NAMESPACE}fill")
            
            # Verificar si el color del resaltado es uno de los que se desea encontrar
            if shading_fill in colores_a_buscar and shading_fill not in colores_encontrados:
                colores_encontrados[shading_fill] = {
                    "archivo": filename,
                    "versión": current_version,
                    "texto": run.text.strip()
                }

                # Finalizar si ya se encontraron todos los colores requeridos
                if len(colores_encontrados) == len(colores_a_buscar):
                    return

# Recorrer todos los archivos .docx en la carpeta
for filename in os.listdir(folder):
    if filename.endswith(".docx") and re.match(r"E\d+", filename):
        file_path = os.path.join(folder, filename)
        procesar_archivo(file_path, filename)

        # Detener la búsqueda si ya se encontraron todos los colores
        if len(colores_encontrados) == len(colores_a_buscar):
            break

# Imprimir los resultados encontrados con su archivo, versión y texto correspondiente
for color, info in colores_encontrados.items():
    print(f"Color {color} encontrado en {info['archivo']}, versión {info['versión']}, texto: '{info['texto']}'")


In [ ]:
# Definir nuevos colores identificados manualmente tras el análisis
colores_nuevos = {
    "azul p": "00b0f0",
    "violeta p": "b08cce",
    "rosa p": "df92ba",
    "color a eliminar": "ffe6e6"  # Este color no se usará, pero se registra para control
}

# Agregar los nuevos colores al diccionario principal
diccionario_colores.update(colores_nuevos)

# Mostrar el diccionario actualizado con todos los colores disponibles
print("Diccionario de colores actualizado:")
for nombre, codigo in diccionario_colores.items():
    print(f"{nombre}: {codigo}")


Diccionario de colores actualizado:
rojo 1: ea9999
rojo 2: e06666
rojo 3: cc0000
naranja 1: f9cb9c
naranja 2: f6b26b
naranja 3: e69138
amarillo 1: ffe599
amarillo 2: ffd966
amarillo 3: f1c232
verde 1: b6d7a8
verde 2: 93c47d
verde 3: 6aa84f
azul 1: a2c4c9
azul 2: 76a5af
azul 3: 45818e
azul 4: a4c2f4
azul 5: 6d9eeb
azul 6: 3c78d8
azul 7: 9fc5e8
azul 8: 6fa8dc
azul 9: 3d85c6
violeta 1: b4a7d6
violeta 2: 8e7cc3
violeta 3: 674ea7
rosa 1: d5a6bd
rosa 2: c27ba0
rosa 3: a64d79
sin resaltado: None
gris 1: 999999
gris 2: cccccc
gris 3: b7b7b7
azul p: 00b0f0
violeta p: b08cce
rosa p: df92ba
color a eliminar: ffe6e6


In [ ]:
# Identificar tipos de formateo presentes en un documento (en este bloque hice pruebas para ver cómo identificar 
# los distintos formateos, estaba experimentando con las funciones para ver qué funcionaba y qué no)

# Definir la ruta del documento a analizar (reemplazar por la ruta correspondiente)
doc_path = "ruta/a/tu/documento.docx"

# Cargar el documento
doc = Document(doc_path)

# Definir flags para registrar los tipos de formateo encontrados
found_bold = False
found_italic = False
found_underline = False
found_strike = False

# Recorrer los párrafos y sus fragmentos (runs) para detectar formateos
for para in doc.paragraphs:
    for run in para.runs:
        if run.bold:
            found_bold = True
        if run.italic:
            found_italic = True
        if run.underline:
            found_underline = True
        if run.font.strike:
            found_strike = True

# Mostrar los resultados encontrados
print("Formateos encontrados en el documento:")
print(f" - Negrita: {'Sí' if found_bold else 'No'}")
print(f" - Itálica: {'Sí' if found_italic else 'No'}")
print(f" - Subrayado: {'Sí' if found_underline else 'No'}")
print(f" - Tachado: {'Sí' if found_strike else 'No'}")


Formateos encontrados en el documento:
 - Negrita: No
 - Itálica: Sí
 - Subrayado: Sí
 - Tachado: Sí


In [ ]:
# Diccionario de colores con sus significados. En este punto ya tenía identificados los colores que aparecían
# en los textos resaltados manunalmente, y había mirado los contextos de aquellos que tenían un marcado ambigüo,
# así que construí este diccionario

significados_colores = {
    "ea9999": "ALT_PUNT",  # rojo 1 
    "e06666": "ALT_PUNT",  # rojo 2 
    "cc0000": "ALT_PUNT",  # rojo 3 

    "f9cb9c": "REEMPLAZO",  # naranja 1   
    "f6b26b": "REEMPLAZO",  # naranja 2   
    "e69138": "REEMPLAZO",  # naranja 3   

    "ffe599": "REORDENAMIENTO",  # amarillo 1   
    "ffd966": "REORDENAMIENTO",  # amarillo 2   
    "f1c232": "REORDENAMIENTO",  # amarillo 3 →  

    "b6d7a8": "REFORMULACION",  # verde 1   
    "93c47d": "REFORMULACION",  # verde 2   
    "6aa84f": "REFORMULACION",  # verde 3   

    "a2c4c9": "ALT_GM",  # azul 1  
    "76a5af": "ALT_GM",  # azul 2  
    "45818e": "ALT_GM",  # azul 3   
    "a4c2f4": "ALT_GM",  # azul 4   
    "6d9eeb": "ALT_GM",  # azul 5   
    "3c78d8": "ALT_GM",  # azul 6   
    "9fc5e8": "ALT_GM",  # azul 7   
    "6fa8dc": "ALT_GM",  # azul 8   
    "3d85c6": "ALT_GM",  # azul 9   
    "00b0f0": "ALT_GM",  #azul p

    "b4a7d6": "INSERCION",  # violeta 1   
    "8e7cc3": "INSERCION",  # violeta 2  
    "674ea7": "INSERCION",  # violeta 3   
    "b08cce": "INSERCION",  # violeta p

    "d5a6bd": "REFORMULACIÓN/REEMPLAZO+REORDENAMIENTO",  # rosa 1
    "c27ba0": "REFORMULACIÓN/REEMPLAZO+REORDENAMIENTO",  # rosa 2
    "a64d79": "REFORMULACIÓN/REEMPLAZO+REORDENAMIENTO",  # rosa 3
    "df92ba": "REFORMULACIÓN/REEMPLAZO+REORDENAMIENTO",  #rosa p

    "999999": "RECUPERACION",  # gris 1   
    "cccccc": "RECUPERACION",  # gris 2
    "b7b7b7": "RECUPERACION"   # gris 3
}

# Quizás acá también debería haber unificado los colores de resaltados de los textos mediante un script auxiliar,
# pero decidí no hacerlo porque en principio solo iba a usar esto para trabajar con el conjunto de datos de mi tesina
# y ya, por lo que prioricé seguir trabajando en lo que seguía del código para registrar las intervenciones

In [ ]:
# Lista de versiones
todas_las_versiones = {"V1","V2", "V3", "V4", "V5", "V6","T1","T2","T3","T4","T5"}

# Filtrar por las versiones que quiero considerar
versiones_validas = {"V2", "V3", "V4", "V5", "V6"}

# Lista donde guardar los datos
data = []

# ID autoincremental para las intervenciones
id_int = 1  

In [ ]:
# Procesar un archivo Word para extraer intervenciones marcadas por color o formato

def extraer_intervenciones(file_path, filename):
    global id_int
    doc = Document(file_path)
    current_version = None  # Almacena la versión actual
    
    for para in doc.paragraphs:
        texto_para = para.text.strip()
        
        # Si el párrafo indica una versión válida, la actualizamos
        if texto_para in todas_las_versiones:
            current_version = texto_para
            continue  # No procesamos la línea de la versión
        
        if current_version:  # Solo consideramos si hay versión
            texto_acumulado = ""
            tipo_actual = None
            
            for run in para.runs:
                shading_fill = None
                rpr = run._element.find(qn('w:rPr'))
                if rpr is not None:
                    shd = rpr.find(qn('w:shd'))
                    if shd is not None:
                        shading_fill = shd.get(qn('w:fill'))
                
                color = shading_fill if shading_fill else None  # Si no tiene resaltado, es None
                texto = run.text.strip()
                
                # Determinar tipo según el color
                if color is not None:
                    if color not in {"999999", "cccccc", "b7b7b7"}:
                        tipo = significados_colores.get(color, "OTRO")
                    elif run.italic:
                        tipo = "RECUPERACION CON REEMPLAZO"
                    else:
                        tipo = "RECUPERACION"
                elif run.font.strike:
                    tipo = "ELIMINACION"
                else:
                    continue  # Ignoramos texto sin intervención
                
                # Si el tipo es el mismo que el anterior, acumulamos el texto
                if tipo == tipo_actual:
                    texto_acumulado += " " + texto
                else:
                    # Guardamos el texto acumulado anterior si existe
                    if texto_acumulado:
                        data.append([id_int, filename.replace(".docx", ""), current_version, texto_acumulado.strip(), tipo_actual])
                        id_int += 1
                    
                    # Reiniciamos acumulación con el nuevo tipo
                    texto_acumulado = texto
                    tipo_actual = tipo
            
            # Guardar el último fragmento acumulado
            if texto_acumulado:
                data.append([id_int, filename.replace(".docx", ""), current_version, texto_acumulado.strip(), tipo_actual])
                id_int += 1


In [ ]:
# Recorrer todos los archivos en la carpeta y procesar con extraer_intervenciones
for filename in os.listdir(folder):
    if filename.endswith(".docx") and re.match(r"E\d+", filename):
        if filename.startswith("E08"):
            continue  # Omitimos el archivo E08
        file_path = os.path.join(folder, filename)
        extraer_intervenciones(file_path, filename)

In [ ]:
# Convertir a DataFrame de Pandas
df = pd.DataFrame(data, columns=["ID_INT", "ID_EXP", "VER", "INTERVENCIÓN", "TIPO"])

# Filtrar solo las versiones válidas (V2, V3, V4, V5, V6)
df = df[df["VER"].isin(versiones_validas)]

# Guardar en CSV
df.to_csv("intervenciones.csv", index=False, encoding="utf-8")

      ID_INT ID_EXP VER                                       INTERVENCIÓN  \
0          1    E00  V2                                           handling   
1          2    E00  V2  all of this has been carried out through the p...   
2          3    E00  V2                                          vegetable   
3          4    E00  V2                                   At the same time   
4          5    E00  V2  Although there were altercations and results t...   
...      ...    ...  ..                                                ...   
1175    1176    E19  V6                                                  .   
1176    1177    E19  V6  indicating that most of the precipitate corres...   
1177    1178    E19  V6  With the completion of the work in the laborat...   
1178    1179    E19  V6                         On the other hand achieved   
1179    1180    E19  V6  despite the possible mistakes that could have ...   

                                        TIPO  
0               